In [ ]:
%pip install transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 88.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [ ]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile

zip_file_path = '/content/gdrive/MyDrive/vison_data/archive.zip'
extract_dir = '/content/vison_data'
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"File extracted to: {extract_dir}")

File extracted to: /content/vison_data


In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests
import copy
import torch
%matplotlib inline

In [ ]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, torch_dtype='auto').eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

processing_florence2.py:   0%|          | 0.00/48.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-large:
- processing_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
def run_example(task_prompt,image, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to('cuda', torch.float16)
    generated_ids = model.generate(
      input_ids=inputs["input_ids"].cuda(),
      pixel_values=inputs["pixel_values"].cuda(),
      max_new_tokens=1024,
      early_stopping=False,
      do_sample=False,
      num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

In [ ]:
image_folder = "/content/vison_data/flickr30k_images/flickr30k_images"
subset_percentage = 30
task_prompt = '<OD>'

In [ ]:
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

In [ ]:
import os
import json
from PIL import Image
from tqdm import tqdm
import random

In [ ]:
data = []
if subset_percentage < 100:
    num_images_to_process = int(len(image_files) * (subset_percentage / 100))
    my_image_files = random.sample(image_files, num_images_to_process)


for filename in tqdm(my_image_files, desc="Processing Images"):
    filepath = os.path.join(image_folder, filename)
    image = Image.open(filepath)

    try:
        parsed = run_example(task_prompt, image)
        data.append(parsed)
    except Exception as e:
        print(f"Skipping {filename} due to error: {e}")
        continue

Processing Images: 100%|██████████| 9534/9534 [2:30:59<00:00,  1.05it/s]


In [ ]:

file_path = '/content/gdrive/MyDrive/vison_data/processed_data.json'
with open(file_path, 'w') as f:
    json.dump(data, f)

print(f"Data saved to {file_path}")


Data saved to /content/gdrive/MyDrive/vison_data/processed_data.json


In [ ]:

file_path = '/content/gdrive/MyDrive/vison_data/images.json'
with open(file_path, 'w') as f:
    json.dump(my_image_files, f)

print(f"my_image_files saved to {file_path}")


my_image_files saved to /content/gdrive/MyDrive/vison_data/images.json


In [ ]:
import os
import json
from PIL import Image
from tqdm import tqdm
import json
import os
from PIL import Image

def build_coco_from_images_and_outputs(images_json, outputs_json, image_dir, output_path):
    with open(images_json, "r") as f:
        image_names = json.load(f)

    with open(outputs_json, "r") as f:
        outputs = json.load(f)

    assert len(image_names) == len(outputs), "Mismatch between number of images and outputs."

    categories = [{"id": 1, "name": "human"}, {"id": 2, "name": "pet"}]
    category_map = {"human": 1, "pet": 2}

    HUMAN_TERMS = [
      "man", "woman", "boy", "girl", "child", "children", "person", "people", "lady",
      "gentleman", "guy", "guys", "worker", "businessman", "craftsman", "cameraman", "showgirl",
      "schoolgirl", "schoolboy", "fireman", "fisherman", "waitperson", "salesman"
    ]
    PET_TERMS = [
      "dog", "puppy", "puppies", "dogs", "cat", "kitten", "kittens", "cats", "horse",
      "horses", "pony", "racehorse", "sheepdog", "doberman"
    ]

    def classify(label):
        label = label.lower()
        if any(h in label for h in HUMAN_TERMS):
            return "human"
        if any(p in label for p in PET_TERMS):
            return "pet"
        return None

    images = []
    annotations = []
    image_id = 1
    annotation_id = 1

    for filename, output in zip(image_names, outputs):
        file_path = os.path.join(image_dir, filename)
        try:
            with Image.open(file_path) as img:
                width, height = img.size
        except Exception as e:
            print(f"Skipping {filename}: {e}")
            continue

        images.append({
            "id": image_id,
            "file_name": filename,
            "width": width,
            "height": height
        })

        od = output.get("<OD>", {})
        bboxes = od.get("bboxes", [])
        labels = od.get("labels", [])

        if not isinstance(bboxes, list) or not isinstance(labels, list):
            continue

        for bbox, label in zip(bboxes, labels):
            class_name = classify(label)
            if not class_name:
                continue

            x1, y1, x2, y2 = bbox
            bbox_coco = [x1, y1, x2 - x1, y2 - y1]
            area = bbox_coco[2] * bbox_coco[3]

            annotations.append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": category_map[class_name],
                "bbox": bbox_coco,
                "area": area,
                "iscrowd": 0
            })
            annotation_id += 1

        image_id += 1

    coco_output = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(output_path, "w") as f:
        json.dump(coco_output, f, indent=2)

    print(f"✅ Saved COCO dataset with {len(images)} images and {len(annotations)} annotations to {output_path}")


In [ ]:
build_coco_from_images_and_outputs("/content/gdrive/MyDrive/vison_data/images.json","/content/gdrive/MyDrive/vison_data/processed_data.json",
                                  "/content/vison_data/flickr30k_images/flickr30k_images","/content/gdrive/MyDrive/vison_data/coco_output.json")

✅ Saved COCO dataset with 9534 images and 45278 annotations to /content/gdrive/MyDrive/vison_data/coco_output.json
